In [38]:
# Choose your fighters
import PyPDF2
import os
from sentence_transformers import SentenceTransformer, util, models
from string2string.alignment import SmithWaterman
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import os
import openai
import pickle
import numpy as np
import configparser
import os
from getpass import getpass

In [19]:
def get_api_key(api_name):
    config_file_path = os.path.expanduser("~/.notebook-api-keys")
    config = configparser.ConfigParser()
    
    # Try reading the existing config file
    if os.path.exists(config_file_path):
        config.read(config_file_path)
    
    # Check if API key is present
    if config.has_option("API_KEYS", api_name):
        return config.get("API_KEYS", api_name)
    
    # If not, prompt the user for the API key
    api_key = getpass(f"Enter your {api_name} API key: ")
    
    # Save the API key in the config file
    if not config.has_section("API_KEYS"):
        config.add_section("API_KEYS")
    config.set("API_KEYS", api_name, api_key)
    
    with open(config_file_path, "w") as f:
        config.write(f)
    
    return api_key

api_key = get_api_key("OpenAI")

In [41]:
cwd = os.getcwd()
states = ['KY', 'CO', 'FL', 'MN', 'TX', 'SD', 'KS', 'NV', 'TN', 'NC', 'SC', 'MN']
sbert = SentenceTransformer('all-mpnet-base-v2') #all-MiniLM-L12-v2 
print(f"SBERT max sequence length: {sbert.max_seq_length}")

sw = SmithWaterman(
    match_weight=1, # weight for a match
    mismatch_weight=0, # weight for a mismatch
    gap_weight=0, # weight for a gap
    gap_char='-' # character to use for a gap
)

384


In [22]:
# https://stackoverflow.com/questions/46975929/how-can-i-calculate-the-jaccard-similarity-of-two-lists-containing-strings-in-py

# TODO swap with Tversky index ??
def jaccard_similarity(list1, list2):
    s1 = set(list1.split())
    s2 = set(list2.split())
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

# Use ChatGPT to Clean and Segment Bills

In [24]:
class ChatGPTWrapper:
    def __init__(self, api_key, model_name="gpt-4-1106-preview"): #"gpt-3.5-turbo"):
        """
        Currently only handles single message sessions.
        """
        self.model_name = model_name
        openai.api_key = api_key
   
    def sample(self, text, temperature=0.8, max_tokens=1000):
        metadata = {"model": self.model_name,
          "temperature": temperature,
          "max_tokens": max_tokens}

        # sample from the openai model
        r = openai.ChatCompletion.create(
          messages = [
            {"role": "system", 
             "content" : """
            Separate the given text into two Python lists.
            List 1 should contain: Section labels (e.g. SECTION 1., §2),Subsection labels (e.g. (a), , A., i, 1.), 
            Summarizing headers (e.g. Bill Summary), 
            Boilerplate language (e.g. Be it enacted by...),
            URLs
            
            List 2 should contain:
            The bill summary sentences (e.g. 'Under current law, a person is subject to a claim of discrimination in places of public accommodation if the person restricts access to a sex-segregated place of public accommodation to individuals based on their biological sex.' 'The bill permits a person to restrict access to a sex-segregated locker room based on an individual’s actual, biological sex.'),
            All other substantive bill content.
            The focus should be on accurately categorizing boilerplate elements in List 1 and substantive content in List 2. 
            Do not add, remove or modify any text. 

            Return as two python lists in this format:
            list1 = ["section 1", "A", "NRS 695I.300", ..., "(A)"]
            list2 = ["There is a longstanding history of restricting access to single-sex public facilities on the basis of sex.", ...]
            """},
            {"role": "user", "content": text}
          ],
          **metadata
        )
        return r['choices'][0]["message"]["content"]
    
wrapper = ChatGPTWrapper(api_key)

In [5]:
# text = "Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite sex not to be present in those circumstances. Parents have a reasonable expectation that public schools in this State will not allow their minor children to be viewed in various states of undress by members of the opposite sex, nor allow their minor children to view members of the opposite sex in various states of undress. Public schools in this State have a duty to protect the dignity, health and welfare of the students in their care. Allowing students to use restrooms, locker rooms, and showers that are reserved for students of a different sex will create a significant potential for disruption of school activities and unsafe situations. Allowing students to use restrooms, locker rooms, and showers that are reserved for students of a different sex will create potential embarrassment, shame, and psychological injury to students. Public schools have a duty to respect and protect the privacy rights of their students. To further the State's interest in protecting all students in public schools in this State; To provide for the privacy needs of all students in public schools in this State; and to maintain order and dignity in restrooms, locker rooms, showers, and other facilities where students may be in various states of undress in the presence of other students. 'Sex' means the physical condition of being male or female, which is determined by a person's chromosomes, and is identified at birth by a person's anatomy. 'Public school' means a school subject to [INSERT RELEVANT CODE PROVISION AUTHORIZING THE CREATION AND SUPERVISION OF PUBLIC SCHOOLS]. Every public school student restroom, locker room, and shower room accessible by multiple students at the same time shall be designated for use by male students only or female students only. In all public schools in this State, student restrooms, locker rooms, and showers that are designated for one sex shall be used only by members of that sex. In any other public school facility or setting where a student may be in a state of undress in the presence of other students, school personnel shall provide separate, private areas designated for use by students based on their sex. Students who consistently assert to school officials that their gender is different from their sex, and whose parent or legal guardian provides written consent to school officials, shall be provided with the best available accommodation, but in no event shall that accommodation be access to student restrooms, locker rooms, or shower rooms designated for use by students of the opposite sex while students of the opposite sex are present or could be present. Acceptable accommodations may include, but are not limited to: access to single-stall bathrooms; access to uni-sex bathrooms; or controlled use of faculty bathrooms, locker rooms, or shower rooms. Students who access a public school restroom, locker room, or shower room designated for use by their sex have a right not to encounter a person of the opposite sex. Students aggrieved under this section who prevail in court may recover from the offending public school $2,500 for each instance in which they encountered a person of the opposite sex while accessing a public school student restroom, locker room, or shower room designated for use by the aggrieved students' sex. They may also recover monetary damages from the offending public school for all psychological, emotional, and physical harm suffered. Nothing in this Section shall limit other remedies at law or equity available to the aggrieved student against the school."
# wrapper = ChatGPTWrapper(api_key)
# result = wrapper.sample(text)

In [17]:
subdir = os.path.join(cwd, "bills")
responses = {}

for bill in os.listdir(subdir)[:2]: # TODO currently just running 2 bills
    filepath = os.path.join(subdir, bill)
    filename, extension = os.path.splitext(filepath)
    with open(filepath, "rb") as file:
        if extension == ".pdf":
            reader = PyPDF2.PdfReader(file, strict=False)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
                
        elif extension == ".html":
            soup = BeautifulSoup(file, "html.parser")
            text = soup.get_text(strip=True)
        else:
            print(f"{filepath}: Unsupported file format")
        result = wrapper.sample(text)
        responses[filename]=result
        

In [20]:
print(responses['NV-AB375.pdf'])

List 1:
- Reprinted with amendments adopted on April 16, 2015
- FIRST REPRINT A.B. 375
- *AB375_R1*
- ASSEMBLY BILL NO. 375–A SSEMBLYWOMAN DOOLING
- MARCH 17, 2015
- JOINT SPONSOR : SENATOR HAMMOND
- Referred to Committee on Education
- SUMMARY—Revises certain provisions concerning public schools. (BDR 34-806)
- FISCAL NOTE: Effect on Local Government: May have Fiscal Impact. Effect on the State: Yes.
- EXPLANATION – Matter in bolded italics is new; matter between brackets [omitted material] is material to be omitted.
- AN ACT relating to education; providing certain requirements for school facilities that are designated for use by persons of one biological sex; and providing other matters properly relating thereto.
- Legislative Counsel’s Digest:
- Section 1 of this bill requires that any school facility in a public school, 1 including a restroom, locker room or shower which is designated for use by persons 2 of one biological sex must only be used by persons of that biological sex, a

# Compare All Bills to ADF's Student Physical Privacy Act

In [25]:
# Step 1: Save ADF's SPPA model legislation 

sppa = [
"Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite sex not to be present in those circumstances.",
"Parents have a reasonable expectation that public schools in this State will not allow their minor children to be viewed in various states of undress by members of the opposite sex, nor allow their minor children to view members of the opposite sex in various states of undress.",
"Public schools in this State have a duty to protect the dignity, health and welfare of the students in their care.",
"Allowing students to use restrooms, locker rooms, and showers that are reserved for students of a different sex will create a significant potential for disruption of school activities and unsafe situations.",
"Allowing students to use restrooms, locker rooms, and showers that are reserved for students of a different sex will create potential embarrassment, shame, and psychological injury to students.",
"Public schools have a duty to respect and protect the privacy rights of their students.",
"To further the States interest in protecting all students in public schools in this State;",
"To provide for the privacy needs of all students in public schools in this State; and to maintain order and dignity in restrooms, locker rooms, showers, and other facilities where students may be in various states of undress in the presence of other students.",
"Sex means the physical condition of being male or female, which is determined by a persons chromosomes, and is identified at birth by a persons anatomy.",
"Public school means a school subject to [INSERT RELEVANT CODE PROVISION AUTHORIZING THE CREATION AND SUPERVISION OF PUBLIC SCHOOLS].",
"Every public school student restroom, locker room, and shower room accessible by multiple students at the same time shall be designated for use by male students only or female students only.",
"In all public schools in this State, student restrooms, locker rooms, and showers that are designated for one sex shall be used only by members of that sex.",
"In any other public school facility or setting where a student may be in a state of undress in the presence of other students, school personnel shall provide separate, private areas designated for use by students based on their sex.",
"Students who consistently assert to school officials that their gender is different from their sex, and whose parent or legal guardian provides written consent to school officials, shall be provided with the best available accommodation, but in no event shall that accommodation be access to student restrooms, locker rooms, or shower rooms designated for use by students of the opposite sex while students of the opposite sex are present or could be present.",
"Acceptable accommodations may include, but are not limited to: access to single-stall bathrooms; access to uni-sex bathrooms; or controlled use of faculty bathrooms, locker rooms, or shower rooms.",
"Students who access a public school restroom, locker room, or shower room designated for use by their sex have a right not to encounter a person of the opposite sex.",
"Students aggrieved under this section who prevail in court may recover from the offending public school $2,500 for each instance in which they encountered a person of the opposite sex while accessing a public school student restroom, locker room, or shower room designated for use by the aggrieved students sex.",
"They may also recover monetary damages from the offending public school for all psychological, emotional, and physical harm suffered.",
"Nothing in this Section shall limit other remedies at law or equity available to the aggrieved student against the school."
]

TX_HB2801 = [
"AN ACT TO provide for single‐sex multiple occupancy bathroom and changing facilities in schools and public agencies and to CREATE statewide consistency in regulation of employment and public ACCOMMODATIONS.",
"Multiple occupancy bathroom or changing facility.",
"Single occupancy bathroom or changing facility.",
"Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.",
"For custodial purposes.",
"For maintenance or inspection purposes.",
"To render medical assistance.",
"To accompany a student needing assistance when the assisting individual is an employee or authorized volunteer of the local board of education or the students parent or authorized caregiver.",
"To receive assistance in using the facility.",
"To accompany a person other than a student needing assistance.",
"That has been temporarily designated for use by that persons biological sex.",
"Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.",
"Executive branch agency.",
"Multiple occupancy bathroom or changing facility.",
"Single occupancy bathroom or changing facility. ",
"Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.",
"Statewide Consistency in laws related to Employment and contracting",
"The provisions of this Article supersede and preempt any ordinance, regulation, resolution, or policy adopted or imposed by a unit of local government or other political subdivision of the State that regulates or imposes any requirement upon an employer pertaining to compensation of employees, such as the wage levels of employees, hours of labor, payment of earned wages, benefits, leave, or well‐being of minors in the workforce.",
"local governments preempted.",
"A city may not require a private contractor under this section to abide by regulations or controls on the contractors employment practices or mandate or prohibit the provision of goods, services, or accommodations to any member of the public as a condition of bidding on a contract or a qualification‐based selection, except as otherwise required by State law.",
"A county may not require a private contractor under this section to abide by regulations or controls on the contractors employment practices or mandate or prohibit the provision of goods, services, or accommodations to any member of the public as a condition of bidding on a contract or a qualification‐based selection, except as otherwise required by State law.",
"PROTECTION OF RIGHTS IN EMPLOYMENT AND PUBLIC ACCOMMODATIONS",
"It is the public policy of this State to protect and safeguard the right and opportunity of all persons to seek, obtain and hold employment without discrimination or abridgement on account of race, religion, color, national origin, age, biological sex or handicap by employers which regularly employ 15 or more employees.",
]

NC_HB2 = [
"AN ACT TO provide for single‐sex multiple occupancy bathroom and changing facilities in schools and public agencies and to CREATE statewide consistency in regulation of employment and public ACCOMMODATIONS.",
"Multiple occupancy bathroom or changing facility.",
"Single occupancy bathroom or changing facility.",
"Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.",
"For custodial purposes.",
"For maintenance or inspection purposes.",
"To render medical assistance.",
"To accompany a student needing assistance when the assisting individual is an employee or authorized volunteer of the local board of education or the students parent or authorized caregiver.",
"To receive assistance in using the facility.",
"To accompany a person other than a student needing assistance.",
"That has been temporarily designated for use by that persons biological sex.",
"Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.",
"Executive branch agency.",
"Multiple occupancy bathroom or changing facility.",
"Single occupancy bathroom or changing facility. ",
"Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.",
"Statewide Consistency in laws related to Employment and contracting",
"The provisions of this Article supersede and preempt any ordinance, regulation, resolution, or policy adopted or imposed by a unit of local government or other political subdivision of the State that regulates or imposes any requirement upon an employer pertaining to compensation of employees, such as the wage levels of employees, hours of labor, payment of earned wages, benefits, leave, or well‐being of minors in the workforce.",
"local governments preempted.",
"A city may not require a private contractor under this section to abide by regulations or controls on the contractors employment practices or mandate or prohibit the provision of goods, services, or accommodations to any member of the public as a condition of bidding on a contract or a qualification‐based selection, except as otherwise required by State law.",
"A county may not require a private contractor under this section to abide by regulations or controls on the contractors employment practices or mandate or prohibit the provision of goods, services, or accommodations to any member of the public as a condition of bidding on a contract or a qualification‐based selection, except as otherwise required by State law.",
"PROTECTION OF RIGHTS IN EMPLOYMENT AND PUBLIC ACCOMMODATIONS",
"It is the public policy of this State to protect and safeguard the right and opportunity of all persons to seek, obtain and hold employment without discrimination or abridgement on account of race, religion, color, national origin, age, biological sex or handicap by employers which regularly employ 15 or more employees.",
]

SC_SB1203 = ['TO AMEND SECTION 45-9-10 OF THE 1976 CODE, RELATING TO THE RIGHT TO EQUAL ENJOYMENT OF AND PRIVILEGES TO PUBLIC ACCOMMODATIONS, TO ADD A SUBSECTION TO PROVIDE THAT UNITS OF LOCAL GOVERNMENT IN THIS STATE MAY NOT ENACT LOCAL LAWS, ORDINANCES, ORDERS, OR OTHER REGULATIONS THAT REQUIRE A PLACE OF PUBLIC ACCOMMODATION OR A PRIVATE CLUB OR OTHER ESTABLISHMENT NOT IN FACT OPEN TO THE GENERAL PUBLIC TO ALLOW A PERSON TO USE A MULTIPLE OCCUPANCY BATHROOM OR CHANGING FACILITY REGARDLESS OF THE PERSONS BIOLOGICAL SEX;',
             'TO AMEND TITLE 10, CHAPTER 1 OF THE 1976 CODE, RELATING TO GENERAL PROVISIONS CONCERNING PUBLIC BUILDINGS AND PROPERTY, BY ADDING ARTICLE 3 TO PROVIDE THAT MULTIPLE OCCUPANCY BATHROOMS AND CHANGING FACILITIES LOCATED ON PUBLIC PROPERTY SHALL BE DESIGNATED FOR AND ONLY USED BY A PERSON BASED ON HIS BIOLOGICAL SEX;' 
'TO AMEND CHAPTER 23, TITLE 59 OF THE 1976 CODE, RELATING TO SCHOOL BUILDINGS AND OTHER SCHOOL PROPERTY, TO PROVIDE THAT LOCAL SCHOOL BOARDS SHALL REQUIRE EVERY MULTIPLE OCCUPANCY BATHROOM OR CHANGING FACILITY THAT IS DESIGNATED FOR STUDENT USE TO BE DESIGNATED FOR AND USED ONLY BY STUDENTS BASED ON THEIR BIOLOGICAL SEX, AND TO PROVIDE EXEMPTIONS AND DEFINITIONS.', 
"Indicates Matter Stricken",'Indicates New Matter',
"Biological sex means the physical condition of being male or female, which is stated on a persons birth certificate.", 
'"Multiple occupancy bathroom" or "changing facility" means a facility designed or designated to be used by more than one person at a time where persons may be in various states of undress in the presence of other persons. A multiple occupancy bathroom or changing facility may include, but is not limited to, a restroom, locker room, changing room, or shower room.', '"Single occupancy bathroom" or "changing facility" means a facility designed or designated to be used by only one person at a time where students may be in various states of undress. A single occupancy bathroom or changing facility may include, but is not limited to, a single stall restroom designated as unisex or for use based on biological sex.', 'Units of local government in this State may not enact local laws, ordinances, orders, or other regulations that require a place of public accommodation or a private club or other establishment not in fact open to the general public to allow a person to use a multiple occupancy bathroom or changing facility regardless of the persons biological sex. A local law, ordinance, order, or other regulation enacted by a unit of local government to require a person to use a multiple occupancy bathroom or changing facility designated for his biological sex is not a violation of this chapter and does not constitute discrimination based upon a protected category.', 'This section does not apply to persons entering a multiple occupancy bathroom or changing facility designated for use by the opposite sex:', 'for custodial purposes;','for maintenance or inspection purposes;','to render medical assistance;','to accompany a person in need of medical assistance;','for a minor under the age of seventeen who accompanies a person caring for that minor;','that has been temporarily designated for use by that persons biological sex."As used in this article:', 'Biological sex" means the physical condition of being male or female, which is stated on a persons birth certificate.', 'Multiple occupancy bathroom or changing facility means a facility designed or designated to be used by more than one person at a time where persons may be in various states of undress in the presence of other persons. A multiple occupancy bathroom or changing facility includes, but is not limited to, a restroom, locker room, changing room, or shower room.', '"Single occupancy bathroom" or "changing facility" means a facility designed or designated to be used by only one person at a time where students may be in various states of undress. A single occupancy bathroom or changing facility includes, but is not limited to, a single stall restroom designated as unisex or for use based on biological sex.', '(A) Multiple occupancy bathrooms and changing facilities located on public property, including but not limited to property owned by the State, its authorities, commissions, departments, committees or agencies, or any political subdivision of the State, shall be designated for and only used by a person based on his biological sex.', '(B) Nothing in this section prohibits public agencies from providing accommodations such as single occupancy bathrooms or changing facilities or controlled use of faculty facilities upon a persons request due to special circumstances. In no event shall such accommodation result in the public agencys allowing a person to use a multiple occupancy bathroom or changing facility as designated in subsection (A) for a sex other than the persons biological sex.', 'This section does not apply to persons entering a multiple occupancy bathroom or changing facility designated for use by the opposite sex:', 'for custodial purposes;','for maintenance or inspection purposes;','to render medical assistance;','to accompany a person in need of medical assistance;','for a minor under the age of seventeen who accompanies a person caring for that minor;','that has been temporarily designated for use by that persons biological sex. As used in this article:', 'Biological sex means the physical condition of being male or female, which is stated on a persons birth certificate.', 'Multiple occupancy bathroom or changing facility means a facility designed or designated to be used by more than one person at a time where students may be in various states of undress in the presence of other persons. A multiple occupancy bathroom or changing facility includes, but is not limited to, a school restroom, locker room, changing room, or shower room.', 'Single occupancy bathroom or changing facility means a facility designed or designated to be used by only one person at a time where students may be in various states of undress. A single occupancy bathroom or changing facility includes, but is not limited to, a single stall restroom designated as unisex or for use based on biological sex.', '(A) Local school boards shall require every multiple occupancy bathroom or changing facility that is designated for student use to be designated for and used only by students based on their biological sex.', '(B) Nothing in this section prohibits a local school board from providing accommodations such as single occupancy bathrooms or changing facilities or controlled use of faculty facilities upon a request due to special circumstances. In no event shall such accommodation result in the local school boards allowing a student to use a multiple occupancy bathroom or changing facility as designated in subsection (A) for a sex other than the students biological sex.', 'This section does not apply to persons entering a multiple occupancy bathroom or changing facility designated for use by the opposite sex:', 'for custodial purposes;','for maintenance or inspection purposes;','to render medical assistance;','to accompany a student needing assistance when the assisting individual is an employee or authorized volunteer of the local school board or the students parent or authorized caregiver;','to receive assistance in using the facility;','to accompany a person other than a student needing assistance;','that has been temporarily designated for use by that persons biological sex."If any section, subsection, paragraph, subparagraph, sentence, clause, phrase, or word of this act is for any reason held to be unconstitutional or invalid, such holding shall not affect the constitutionality or validity of the remaining portions of this act, the General Assembly hereby declaring that it would have passed this Act, and each and every section, subsection, paragraph, subparagraph, sentence, clause, phrase, and word thereof, irrespective of the fact that any one or more other sections, subsections, paragraphs, subparagraphs, sentences, clauses, phrases, or words hereof may be declared to be unconstitutional, invalid, or otherwise ineffective.', 
'This act takes effect upon approval by the Governor.']
KY_SB76 = ['AN ACT relating to student privacy.', "Be it enacted by the General Assembly of the Commonwealth of Kentucky:", '(1) As used in this section:', 'Biological sex means the physical condition of being male or female, which is determined by a persons chromosomes, and is identified at birth, by a persons anatomy;', 
           'and "Public school" or "school" means a school under the control of a local board of education of a local school district.', 
           'Every school restroom, locker room, and shower room designated for student use accessible by multiple students at the same time shall be designated for and used by male students only or female students only.', 
           'In every school setting in which a student may be in a state of undress in the presence of other students, school personnel shall provide separate, private areas designated for use by students based on their biological sex.', 
           'A student who asserts to school officials that his or her gender is different from his or her biological sex and whose parent or legal guardian provides written consent to school officials shall be provided with the best available accommodation, but that accommodation shall not include the use of student restrooms, locker rooms, or shower rooms designated for use by students of the opposite biological sex while students of the opposite biological sex are present or could be present.', 
           'Acceptable accommodations may include but are not limited to access to single-stall restrooms, access to unisex bathrooms, or controlled use of faculty bathrooms, locker rooms, or shower rooms.', 
           'Any portion of this section declared invalid shall not affect the validity of the remaining portions of this section.']
CO_HB1081 = ['Under current law, a person is subject to a claim of discrimination in places of public accommodation if the person restricts access to a sex-segregated place of public accommodation to individuals based on their biological sex.', 'The bill permits a person to restrict access to a sex-segregated locker room based on an individuals actual, biological sex.', 'Be it enacted by the General Assembly of the State of Colorado:', 'The general assembly finds that:', 'Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite biological sex not to be present in those circumstances;', 'Parents have a reasonable expectation that places of public accommodation in this state that have sex-segregated locker rooms will not allow their minor children to be viewed in various states of undress by members of the opposite biological sex, nor allow their minor children to view members of the opposite biological sex in various states of undress;', 'Allowing individuals to use locker rooms that are reserved for individuals of a different biological sex will create a significant potential for unsafe situations and for an invasion of privacy.', 'In Colorado Revised Statutes, -601, amend () as follows:', 'Notwithstanding any other provisions of this section, it is not a discriminatory practice for a person to restrict admission to a place of public accommodation to individuals of one sex if: such', 'T HE restriction has a bona fide relationship to the goods, services, facilities, privileges, advantages, or accommodations of such', 'T HE PLACE OF PUBLIC ACCOMMODATION IS A SEX- SEGREGATED LOCKER ROOM AND THE RESTRICTION IS BASED ON AN INDIVIDUALS ACTUAL, BIOLOGICAL SEX.', 'The general assembly hereby finds, determines, and declares that this act is necessary for the immediate preservation of the public peace, health, and safety.']
SD_HB1008 = ['FOR AN ACT ENTITLED, An Act to restrict access to certain restrooms and locker rooms in public schools.', 'BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF SOUTH DAKOTA:', 'The term, biological sex, as used in this Act, means the physical condition of being male or female as determined by a persons chromosomes and identified at birth by a persons anatomy.', 'Every restroom, locker room, and shower room located in a public elementary or secondary school that is designated for student use and is accessible by multiple students at the same time shall be designated for and used only by students of the same biological sex. In addition, any public school student participating in a school sponsored activity off school premises which includes being in a state of undress in the presence of other students shall use those rooms designated for and used only by students of the same biological sex.', 'If any student asserts that the students gender is different from the students biological sex, and if the students parent or guardian consents to that assertion in writing to a public school administrator, or if the student is an adult or an emancipated minor and makes the assertion in writing to a public school administrator, the student shall be provided with a reasonable accommodation. A reasonable accommodation is one that does not impose an undue hardship on a school district. A reasonable accommodation may not include the use of student restrooms, locker rooms, or shower rooms designated for use by students of the opposite biological sex if students of the opposite biological sex are present or could be present. A reasonable accommodation may include a single-occupancy restroom, a unisex restroom, or the controlled use of a restroom, locker room, or shower room that is designated for use by faculty. The requirement to provide a reasonable accommodation pursuant to this section does not apply to any nonpublic school entity.', 'If any public school district, school district officer or employee, school board, or school board member is sued in state or federal court as a result of a decision based upon and consistent with a students biological sex, notwithstanding any assertion that the students gender is different than the students biological sex, the attorney general shall represent the school district, school district officer or employee, school board, or school board member at no cost to the school district, school district officer or employee, school board, or school board member, and the State of South Dakota shall assume all financial responsibility for the legal expenses. The legal expenses for which the state is responsible include any award for monetary damages or attorneys fees and costs which may be awarded and for which the school district, school district officer or employee, school board, or school board member would otherwise be responsible.']
FL_HB0583 = ['providing purpose and legislative findings; creating s. , F.S.; providing definitions; requiring that use of single-sex public facilities be restricted to persons of the sex for which the facility is designated; prohibiting knowingly and willfully entering a single-sex public facility designated for or restricted to persons of the other biological sex; providing criminal penalties; providing exemptions; providing private cause of action against violators; providing for preemption; providing an effective date.', 'The purpose of this act is to secure privacy and safety for all individuals using single-sex public facilities.', 'The Legislature finds that:', 'There is a longstanding history of restricting access to single-sex public facilities on the basis of sex.', 'There is an expectation of privacy in single-sex public facilities.', 'Users of single-sex public facilities reasonably expect not to be exposed to individuals of the other sex while using those facilities.', 'Single-sex public facilities are places of increased vulnerability and present the potential for crimes against individuals using those facilities, including, but not limited to, assault, battery, molestation, rape, voyeurism, and exhibitionism.', "Public accommodations has the same meaning provided in s. .", '"Single-sex public facilities means bathrooms, restrooms, dressing rooms, fitting rooms, locker rooms, showers, and other similar facilities where there is a reasonable expectation of privacy; that are maintained by an owner of public accommodations, a school, or a place of employment; that are conspicuously designated with appropriate signage for use by persons of only one sex; and that are designed or designated to be used by more than one person at a time.', 'Sex means a person s biological sex, either male or female, at birth. For purposes of this paragraph, the term means a person born as a biological male and the term means a person born as a biological female.', 'Single-sex public facilities designated for girls, women, ladies, or persons of the female sex shall be restricted to persons who are biological females.', 'Single-sex public facilities designated for boys, men, gentlemen, or persons of the male sex shall be restricted to persons who are biological males.', 'A person who knowingly and willfully enters a single- sex public facility designated for or restricted to persons of the other biological sex commits a misdemeanor of the first degree, punishable as provided in s. or s. .', 'This section does not apply to public facilities that are conspicuously designated for unisex or family use.', 'This section does not apply to public facilities that are designated to be used by only one person at a time.', 'This section does not require any place of public accommodation, school, or place of employment to construct or maintain single-sex public facilities or to modify existing public facilities.', 'Restricting access to single-sex public facilities in the manner required by subsection () may not be deemed unlawful discrimination under s. .', 'A person who knowingly and willfully enters a single- sex public facility designated for the other biological sex is liable in a civil action to any person who is lawfully using the same single-sex public facility at the time of the unlawful entry for the damages caused by the unlawful entry, together with reasonable attorney fees and costs.', 'An owner of public accommodations, a school, or a place of employment who maintains single-sex public facilities and advertises, promotes, or encourages use of those facilities in violation of subsection (), or fails to take reasonable remedial measures after learning of such use, is liable in a civil action to any person who is lawfully using those facilities at the time of the unlawful entry for the damages caused by the unlawful entry, together with reasonable attorney fees and costs.', 'This section preempts any law, regulation, policy, or decree enacted or adopted by any city, county, municipality, or other political subdivision within the state that purports to permit or require owners of public accommodations, schools, or places of employment to permit use of single-sex public facilities by persons whose biological sex is different from the sex for which such facilities are designated.']
TN_HB2414 = ['AN ACT to amend Tennessee Code Annotated, Title 49, relative to student restrooms and locker rooms.', 'Public schools shall require that a student use student restroom and locker room facilities that are assigned for use by persons of the same sex as the sex indicated on the students original birth certificate.', 'Public institutions of higher education shall require that a student use the restroom and locker room facilities that are assigned for use by persons of the same sex as the sex indicated on the student’s original birth certificate.', 'This act shall take effect upon becoming a law, the public welfare requiring it.']
MN_HF0041 = ['A bill for an act relating to education; establishing the Student Physical Privacy Act; proposing coding for new law in Minnesota Statutes, chapter 121A.', 'BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF MINNESOTA:', 'The purpose of this section is to protect and provide for the privacy and safety of all students enrolled in public schools and to maintain order and dignity in restrooms, locker rooms, changing rooms, showers, and other facilities where students may be in various states of undress in the presence of other students.', 'For the purposes of this section, the following terms have the meanings given them.', '"Sex" means the physical condition of being male or female, which is determined by a persons chromosomes and is identified at birth by a persons anatomy.', 'Public school means a public school under section 120A.05, subdivisions 9, 11, 13, and 17, and a charter school under chapter 124E.', 'A public school student restroom, locker room, changing room, and shower room accessible by multiple students at the same time shall be designated for the exclusive use by students of the male sex only or by students of the female sex only.', 'A public school student restroom, locker room, changing room, and shower room that is designated for the exclusive use of one sex shall be used only by members of that sex.', 'In any other public school facility or setting where a student may be in a state of undress in the presence of other students, school personnel shall provide separate, private, and safe areas designated for use by students based on their sex.', 'Nothing in this section shall prohibit public schools from providing accommodation such as single-occupancy facilities or controlled use of faculty facilities upon a student request due to special circumstances, but in no event shall that accommodation result in a public school allowing a student to use a facility designated under paragraph (b) for a sex other than the students own sex.']
KS_HB2737 = ['AN ACT concerning school districts; creating the student physical privacy act.', 'Be it enacted by the Legislature of the State of Kansas:', 'The provisions of sections 1 through 7, and amendments thereto, shall be known and may be cited as the student physical privacy act.', 'The legislature hereby finds and declares:', 'Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite sex not to be present in those circumstances.', 'Parents have a reasonable expectation that public schools in this state will not allow their minor children to be viewed in various states of undress by members of the opposite sex, nor allow their minor children to view members of the opposite sex in various states of undress.', 'Young adults have a reasonable expectation that postsecondary educational institutions in this state will not allow their students to be viewed in various states of undress by members of the opposite sex while using student restrooms, locker rooms and showers.', 'Public schools and postsecondary educational institutions in this state have a duty to protect the dignity, health and welfare of the students in their care.', 'Allowing students to use restrooms, locker rooms and showers that are reserved for students of a different sex will create a significant potential for disruption of educational activities and unsafe situations.', 'Allowing students to use restrooms, locker rooms and showers that are reserved for students of a different sex will create potential embarrassment, shame and psychological injury to students.', 'Public schools and postsecondary educational institutions have a duty to respect and protect the privacy rights of their students. Courts have recognized a constitutional right to privacy that includes a right not to be compelled by the government to undress or be unclothed in the presence of members of the opposite sex.', 'See, e.g., Cumbey v. Meachum, 684 F.2d 712 (10 th Cir. 1982); Lee v. Downs, 641 F.2d 1117 (4t h Cir. 1981);', 'York v. Story, 324 F.2d 450 (9 th Cir. 1963).', 'The purposes of this act are:', 'To further the states interest in protecting all students in public schools and postsecondary educational institutions in this state;', 'To provide for the privacy needs of all students in public schools and postsecondary educational institutions in this state;', 'To maintain order and dignity in restrooms, locker rooms, showers and other facilities where students may be in various states of undress in the presence of other students.', 'Sex means the physical condition of being male or female, which is determined by a persons chromosomes, and is identified at birth by a persons anatomy.', 'Postsecondary educational institution shall have the same meaning as that term is defined in K.S.A. 74-3201b, and amendments thereto.', 'Public school means a school operated by a unified school district organized under the laws of this state.', 'Every public school and postsecondary educational institution student restroom, locker room and shower room accessible by multiple students at the same time shall be designated for use by male students only or female students only.', 'In all public schools and postsecondary educational institutions in this state, student restrooms, locker rooms and showers that are designated for one sex shall be used only by members of that sex.', 'In any other public school facility, postsecondary educational institution facility or setting not specified in subsection where a student may be in a state of undress in the presence of other students, school or institution personnel shall provide separate, private areas designated for use by students based on their sex.', 'Students who, for any reason, desire greater privacy when using a public school or postsecondary educational institution restroom, locker room or shower room, and whose parents or legal guardians provide written consent to school or institution officials, may submit a request to such officials for access to alternative facilities. The school or institution official to whom a request is submitted shall evaluate such request and shall, to the extent reasonable, offer options for alternative facilities. In no event shall any accommodation be an access to student restrooms, locker rooms or shower rooms designated for use by students of the opposite sex while students of the opposite sex are present or could be present.', 'Acceptable accommodations may include, but are not limited to: Access to single-stall bathrooms; access to unisex bathrooms; or controlled use of faculty bathrooms, locker rooms or shower rooms.', 'Students who access a public school or postsecondary educational institution restroom, locker room or shower room designated for use by such students sex have a right not to encounter a person of the opposite sex.', 'Students who, while accessing a public school or postsecondary educational institution student restroom, locker room or shower room designated for use by such students sex, encounter a person of the opposite sex, have a private cause of action against the school district or postsecondary educational institution, if:', 'The public school or postsecondary educational institution gave such person of the opposite sex permission to use facilities designated for use by such students sex;', 'the public school or postsecondary educational institution failed to take reasonable steps to prohibit such person of the opposite sex from using facilities designated for use by such students sex.', 'A cause of action brought pursuant to this section shall be brought in either the state district court or the federal district court for the jurisdiction where either the student resides at the time such action is filed, or where the public school or postsecondary educational institution is located.', 'No cause of action may be brought pursuant to this section more than four years after a violation of subsection occurred.', 'Students aggrieved under this section may obtain appropriate relief, which shall include:', 'Statutory damages in an amount of $2,500 for each instance in which the aggrieved student encountered a person of the opposite sex while accessing a public school or postsecondary educational institution student restroom, locker room or shower room designated for use by the aggrieved students sex;', 'monetary damages for all psychological, emotional and physical harm suffered as a result of a violation of this section;', 'reasonable attorney fees and costs;', 'and such other relief as the court deems appropriate.', 'If any provision of sections 1 through 6, and amendments thereto, or its application to any person or circumstance is held invalid, the invalidity shall not affect any other provision or application of sections 1 through 6, and amendments thereto, which can be given effect without the invalid provision or application. To this end the provisions of sections 1 through 6, and amendments thereto, are severable.', 'This act shall take effect and be in force from and after its publication in the statute book.']
NV_AB375 = ['providing certain requirements for school facilities that are designated for use by persons of one biological sex; and providing other matters properly relating thereto.', 'Any school facility in a public school, including, without limitation, a restroom, locker room or shower which is designated for use by persons of one biological sex must only be used by persons of that biological sex.', 'In any school facility or setting where a pupil may be in a state of undress in the presence of other pupils, a public school shall provide separate, private areas designated for use by pupils based on their biological sex.', 'For any pupil who asserts at school a gender that is different than the pupils biological sex, a public school shall provide the best available accommodation that meets the needs of the pupil, but such accommodation must not include access to a school restroom, locker room or shower designated for use by persons whose biological sex is different from the pupils biological sex. Such accommodation may include, without limitation, access to a single-stall restroom, access to a unisex restroom or controlled use of a faculty restroom, locker room or shower.', 'As used in this section, biological sex means the biological condition of being male or female as determined at birth based on physical differences or, if necessary, at the chromosomal level.', 'Except as otherwise provided in section 1 of this act, all persons are entitled to the full and equal enjoyment of the goods, services, facilities, privileges, advantages and accommodations of any place of public accommodation, without discrimination or segregation on the ground of race, color, religion, national origin, disability, sexual orientation, sex, gender identity or expression.', 'This act becomes effective upon passage and approval for the purpose of adopting any regulations and performing any other preparatory administrative tasks to carry out the provisions of this act, and on January 1, 2016, for all other purposes.']

# MN_SF1543 = 


In [26]:
bills = {"NC_HB2":NC_HB2,"TX_HB2801":TX_HB2801,"SC_SB1203":SC_SB1203,"KY_SB76":KY_SB76,
         "CO_HB1081":CO_HB1081,"SD_HB1008":SD_HB1008,"FL_HB0583":FL_HB0583,"TN_HB2414":TN_HB2414,
         "MN_HF0041":MN_HF0041,"KS_HB2737":KS_HB2737,"NV_AB375":NV_AB375}

In [42]:
comparisons = []
max_len = 0 
for sppa_segment in sppa:
    sppa_embedding = sbert.encode(sppa_segment.lower())
    for bill in bills.keys():
        for bill_segment in bills[bill]:
            # Make sure embeddings are not cropping text segment 
            try:
                o = sbert.tokenizer(bill_segment, return_attention_mask=False, return_token_type_ids=False)
                if len(o["input_ids"]) > sbert.max_seq_length:
                    print(len(o["input_ids"]))
                    raise ValueError("Segment is longer than context window.")
                # Calculate embeddings and cosine similarity
                bill_embedding = sbert.encode(bill_segment.lower())
            except ValueError as e:
                print(e)
            cos_score = util.cos_sim(sppa_embedding.reshape(1, -1), bill_embedding.reshape(1, -1))[0][0]
            
            if len(bill_segment) >= len(sppa_segment):
                max_len = len(bill_segment)
            else:
                max_len = len(sppa_segment)

            # Calculate raw Smith-Waterman score
            sw_score = sw.get_alignment_score(sppa_segment.lower(), bill_segment.lower())/max_len

            # Calculate Jaccard Similiarity
            jaccard = jaccard_similarity(sppa_segment.lower(), bill_segment.lower())

            # Only store if greater than certain value
            if sw_score > 0.4:
                # Store vals in array
                comparisons.append([bill, float(cos_score), sw_score, sppa_segment, bill_segment, jaccard])

In [43]:
for item in comparisons:
    print(f"Cosine Similarity: {item[1]:.4f}")
    print(f"Jaccard: {item[5]:.4f}")
    print(f"S-W:{item[2]}")
    print(f"SPPA segment: {item[3]}")
    print(f"{item[0]} segment: {item[4]}")
    print("-" * 50)

Cosine Similarity: 0.9740
Jaccard: 0.9118
S-W:0.7767857142857143
SPPA segment: Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite sex not to be present in those circumstances.
CO_HB1081 segment: Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite biological sex not to be present in those circumstances;
--------------------------------------------------
Cosine Similarity: 1.0000
Jaccard: 1.0000
S-W:1.0
SPPA segment: Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite sex not to be present in those circumstances.
KS_HB2737 segment: Children and young adults have natural and normal concerns about physical privacy when they are in various states o

In [48]:
# sppa = '''
# Children and young adults have natural and normal concerns about physical privacy when they are in various states of undress, and most wish for members of the opposite sex not to be present in those circumstances.
# Parents have a reasonable expectation that public schools in this State will not allow their minor children to be viewed in various states of undress by members of the opposite sex, nor allow their minor children to view members of the opposite sex in various states of undress.
# Public schools in this State have a duty to protect the dignity, health and welfare of the students in their care.
# Allowing students to use restrooms, locker rooms, and showers that are reserved for students of a different sex will create a significant potential for disruption of school activities and unsafe situations.
# Allowing students to use restrooms, locker rooms, and showers that are reserved for students of a different sex will create potential embarrassment, shame, and psychological injury to students.
# Public schools have a duty to respect and protect the privacy rights of their students.
# To further the State’s interest in protecting all students in public schools in this State;
# To provide for the privacy needs of all students in public schools in this State; and to maintain order and dignity in restrooms, locker rooms, showers, and other facilities where students may be in various states of undress in the presence of other students.
# “Sex” means the physical condition of being male or female, which is determined by a person’s chromosomes, and is identified at birth by a person’s anatomy.
# “Public school” means a school subject to [INSERT RELEVANT CODE PROVISION AUTHORIZING THE CREATION AND SUPERVISION OF PUBLIC SCHOOLS].
# Every public school student restroom, locker room, and shower room accessible by multiple students at the same time shall be designated for use by male students only or female students only.
# In all public schools in this State, student restrooms, locker rooms, and showers that are designated for one sex shall be used only by members of that sex.
# In any other public school facility or setting where a student may be in a state of undress in the presence of other students, school personnel shall provide separate, private areas designated for use by students based on their sex.
# Students who consistently assert to school officials that their gender is different from their sex, and whose parent or legal guardian provides written consent to school officials, shall be provided with the best available accommodation, but in no event shall that accommodation be access to student restrooms, locker rooms, or shower rooms designated for use by students of the opposite sex while students of the opposite sex are present or could be present.
# Acceptable accommodations may include, but are not limited to: access to single-stall bathrooms; access to uni-sex bathrooms; or controlled use of faculty bathrooms, locker rooms, or shower rooms.
# Students who access a public school restroom, locker room, or shower room designated for use by their sex have a right not to encounter a person of the opposite sex.
# Students aggrieved under this section who prevail in court may recover from the offending public school $2,500 for each instance in which they encountered a person of the opposite sex while accessing a public school student restroom, locker room, or shower room designated for use by the aggrieved students’ sex.
# They may also recover monetary damages from the offending public school for all psychological, emotional, and physical harm suffered.
# Nothing in this Section shall limit other remedies at law or equity available to the aggrieved student against the school.
# '''

# NC_HB2 = '''
# AN ACT TO provide for single‐sex multiple occupancy bathroom and changing facilities in schools and public agencies and to CREATE statewide consistency in regulation of employment and public ACCOMMODATIONS.
# Multiple occupancy bathroom or changing facility.
# Single occupancy bathroom or changing facility.
# Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.
# For custodial purposes.
# For maintenance or inspection purposes.
# To render medical assistance.
# To accompany a student needing assistance when the assisting individual is an employee or authorized volunteer of the local board of education or the student's parent or authorized caregiver.
# To receive assistance in using the facility.
# To accompany a person other than a student needing assistance.
# That has been temporarily designated for use by that person's biological sex.
# Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.
# Executive branch agency.
# Multiple occupancy bathroom or changing facility.
# Single occupancy bathroom or changing facility.
# Single‐Sex Multiple Occupancy Bathroom and Changing Facilities.
# Statewide Consistency in laws related to Employment and contracting
# The provisions of this Article supersede and preempt any ordinance, regulation, resolution, or policy adopted or imposed by a unit of local government or other political subdivision of the State that regulates or imposes any requirement upon an employer pertaining to compensation of employees, such as the wage levels of employees, hours of labor, payment of earned wages, benefits, leave, or well‐being of minors in the workforce.
# local governments preempted.
# A city may not require a private contractor under this section to abide by regulations or controls on the contractor's employment practices or mandate or prohibit the provision of goods, services, or accommodations to any member of the public as a condition of bidding on a contract or a qualification‐based selection except as otherwise required by State law.
# A county may not require a private contractor under this section to abide by regulations or controls on the contractor's employment practices or mandate or prohibit the provision of goods, services, or accommodations to any member of the public as a condition of bidding on a contract or a qualification‐based selection except as otherwise required by State law.
# PROTECTION OF RIGHTS IN EMPLOYMENT AND PUBLIC ACCOMMODATIONS
# It is the public policy of this State to protect and safeguard the right and opportunity of all persons to seek, obtain and hold employment without discrimination or abridgement on account of race, religion, color, national origin, age, biological sex or handicap by employers which regularly employ 15 or more employees.
# '''

# TX_HB2801 = '''
# relating to the use of a bathroom, locker room, or shower facility in a public school.
# A school district shall adopt a policy providing that only persons of the same biological sex may be present at the same time in any bathroom, locker room, or shower facility in a building owned by the district.
# A school district shall provide reasonable alternate bathroom, locker room, or shower facility accommodations to a student if: the student asserts a gender identity that is different from the student's biological sex;
# and the student's parent or guardian consents in writing to the provision of alternate accommodations.
# A school district must comply with the policy adopted under Section 38.251 in providing alternate accommodations under Subsection (a).
# This section does not require a school district to construct, maintain, or modify a bathroom, locker room, or shower facility to provide any alternate accommodations required under Subsection (a).
# A school district is liable to a student who encounters a person not of the student's biological sex in a bathroom, locker room, or shower facility that is designated for use by persons of the student's biological sex if any employee of the district
# (1) knew that the person was not of the same biological sex for which the bathroom, locker room, or shower facility was designated;
# and (2) permitted the person to enter or failed to take reasonable steps to prevent the person from entering the bathroom, locker room, or shower facility.
# It is a defense to liability under this section that the person entered the bathroom, locker room, or shower facility for a custodial purpose, for the purpose of repairing the facility, to give medical assistance, or to supervise or assist a student.
# A student who prevails in a suit under this section shall be awarded:
# (1) exemplary damages in the amount of $2,000; (2) actual damages, including damages for mental anguish even if an injury other than mental anguish is not shown;
# (3) court costs; and (4) reasonable attorney's fees.
# The cause of action created by this section is cumulative of any other remedy provided by common law or statute.
# This section does not override the immunity from personal liability granted in Section 22.0511 or other law.
# This subchapter does not prohibit a school district from constructing or maintaining a bathroom, locker room, or shower facility designed for use by only one person at a time or from modifying existing facilities for that use.
# '''